# FractalN Mushroom Classification

This notebook runs the training process in Google Colab using the main.py script.

## Setup
First, let's clone the repository and install dependencies:

In [ ]:
# First check CUDA version
!nvidia-smi
!nvcc --version

# Install the correct CuPy version based on CUDA version
import subprocess
cuda_output = subprocess.getoutput('nvcc --version')
if 'release 11' in cuda_output:
    !pip install cupy-cuda11x
elif 'release 12' in cuda_output:
    !pip install cupy-cuda12x
else:
    !pip install cupy-cuda110  # Fallback for older versions

# Install other requirements
!pip install colorama tqdm pillow numpy opencv-python

# Clone repository
!git clone https://github.com/YOUR_USERNAME/FractalN.git
%cd FractalN

# Verify imports
import cupy as cp
print(f"\nCuPy version: {cp.__version__}")
print(f"CUDA available: {cp.cuda.is_available()}")

## GPU Check
Let's verify GPU availability:

In [ ]:
!nvidia-smi

In [ ]:
# Verify CuPy can access GPU
import cupy as cp

# Test GPU computation
x = cp.array([1, 2, 3])
print(f"CuPy GPU available: {cp.cuda.is_available()}")
if cp.cuda.is_available():
    device = cp.cuda.Device(0)
    print(f"GPU Device: {device.attributes['DEVICE_NAME'].decode('utf-8')}")
    print(f"Total Memory: {device.mem_info[1]/1024**3:.2f} GB")

# Test basic computation
y = cp.square(x)
print(f"\nTest computation on GPU: {x} squared = {y}")
print("GPU test successful!")

## Training
Now we'll run the training using main.py:

In [ ]:
import sys
sys.path.append('src')

from gpu_utils import configure_gpu
from main import train

# Configure GPU with memory pool
configure_gpu()

# Run your custom CNN implementation
train()